In [13]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 기본 테마 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats
import statsmodels as sm
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [14]:
data = pd.read_csv('data/2324.csv')
data.head()
data.columns

C:\Users\Tammy Kim\AppData\Local\Temp\ipykernel_18076\3492930622.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/2324.csv')


Index(['id', 'visitnum', 'triptype', 'purpose', 'act_interest',
       'infra_interest', 'info_1', 'info_2', 'info_3', 'info_na',
       ...
       'nationality', 'gender', 'age_group', 'visit_purpose_cat',
       'visit_count_cat', 'trip_type_cat', 'weight', 'year', 'wellness',
       'infolack'],
      dtype='object', length=105)

In [15]:
# 통제변수
model_df = data[['overall_satis', 'infolack', 'wellness', 'gender', 'age_group', 'purpose', 'visitnum', 'triptype', 'companion' ]].copy()
# 다중 회귀 분석을 위한 결측치 제거
model_df.dropna(inplace=True)
model_df.head()

,overall_satis,infolack,wellness,gender,age_group,purpose,visitnum,triptype,companion
0,5,2,0,2,2,1,4,1,1
1,5,0,1,1,3,2,4,1,1
2,4,0,0,2,4,2,3,1,1
3,5,3,0,2,4,1,1,1,2
4,5,3,1,2,3,1,1,1,2


In [19]:
# 종속변수
y = model_df['overall_satis']

# 상호작용 항 생성
model_df['infolack+wellness'] = model_df['infolack'] * model_df['wellness']

# 독립변수 리스트
X_cols = ['infolack', 'wellness', 'infolack+wellness', 'gender', 'age_group', 'purpose', 'visitnum', 'triptype', 'companion']
X = model_df[X_cols]

# 순서형 로지스틱 회귀 모델
model = OrderedModel(y, X, distr='logit')
results = model.fit(method='bfgs')

# 주요 결과 해석
print("\n--- 주요 변수 결과 해석 ---")
if 'infolack' in results.params:
    print(f"정보 부족 (infolack) 계수: {results.params['infolack']:.3f}, P-value: {results.pvalues['infolack']:.3f}")
if 'wellness' in results.params:
    print(f"웰니스 여부 (wellness) 계수: {results.params['wellness']:.3f}, P-value: {results.pvalues['wellness']:.3f}")
if 'infolack_x_wellness' in results.params:
    print(f"정보 부족 * 웰니스 (상호작용) 계수: {results.params['infolack_x_wellness']:.3f}, P-value: {results.pvalues['infolack_x_wellness']:.3f}")

if 'infolack_x_wellness' in results.pvalues and results.pvalues['infolack_x_wellness'] < 0.05:
    print("\n상호작용 항이 통계적으로 유의미합니다.")
    print("이는 웰니스 그룹에서 정보 부족이 만족도에 미치는 영향이 일반 그룹과 다름을 의미합니다.")
elif 'infolack' in results.pvalues and results.pvalues['infolack'] < 0.05:
    print("\n상호작용 항은 유의미하지 않으나, 정보 부족 (infolack) 자체가 만족도에 유의미한 영향을 미칩니다.")
else:
    print("\n정보 부족과 웰니스 여부, 그리고 그 상호작용이 만족도에 유의미한 영향을 미치지 않습니다.")

Optimization terminated successfully.
         Current function value: 0.736604
         Iterations: 65
         Function evaluations: 66
         Gradient evaluations: 66

--- 주요 변수 결과 해석 ---
정보 부족 (infolack) 계수: -0.100, P-value: 0.000
웰니스 여부 (wellness) 계수: -0.175, P-value: 0.001

상호작용 항은 유의미하지 않으나, 정보 부족 (infolack) 자체가 만족도에 유의미한 영향을 미칩니다.


### logistic regression model 해석
1. 정보 부족 (infolack)의 계수: -0.100 (p < 0.001)
- 정보 부족 점수가 높을수록 전반적인 만족도는 낮아진다는 의미
- 계수가 음수이므로, 정보 부족 경험이 만족도를 저해하는 요인임을 시사

2. 웰니스 여부 (wellness)의 계수: -0.175 (p = 0.001)
- 웰니스 방문자는 비웰니스 방문자에 비해 만족도가 유의미하게 낮다는 결과
- 이는 의외일 수 있지만, 웰니스 관광객들이 더 많은 기대를 하고 방문하거나, 정보 접근성 부족에 더 민감할 가능성을 제시
- 즉, 웰니스 관광의 질에 대한 기대 대비 충족도 불일치 문제일 수 있습니다.
3. 상호작용항 (infolack × wellness, 웰니스 온 사람이 정보가 부족하면 더 실망하는 건지)은 유의미하지 않음
- 웰니스 방문자라고 해서 정보 부족이 만족도에 미치는 영향이 달라지지는 않는다는 뜻
- 즉, 정보 부족은 모든 방문객에게 부정적인 영향을 미치며, 웰니스 여부에 따른 영향력의 차이는 없음.

- 정보 부족 변수는 세 문항의 응답 수에 기반하여 구성되었으나 
일부 응답자에게는 결측치가 포함되어 있어, 측정 오류 발생 위험 존재.
- 더불어 정보 부족의 강도를 완벽히 반영하지 못하는 한계가 있음. 특히 특정 항목 하나만으로 강한 정보 부족을 경험할 수 있는 점을 감안하면 해석에 매우 신중을 가할 필요가 있음.
- 지금 가진 데이터로는 분석의 신뢰성이 있다고 판단할 수 없음.
- 가설을 변경하고, 다른 분석을 해봐야겠다고 판단함. 